In [13]:
OPACITY = 0.5
NUM_IMAGES = 20

In [ ]:
import io
import webbrowser
from PIL import Image
import img2pdf
import os
import requests

def GetImage():
    url = 'https://api.waifu.im/search'
    response = requests.get(url)
    data = response.json()
    response = requests.get(data["images"][0]["url"])
    return response.content

In [ ]:
def CreateTransparentPDF(images, opacity=0.5):
    ProcessedImages = []
    
    for img in images:
        img = Image.open(io.BytesIO(img)).convert("RGBA")
        
        alpha = img.split()[3]
        alpha = alpha.point(lambda p: int(p * opacity))
        img.putalpha(alpha)
        rgb_img = Image.new("RGB", img.size, (255, 255, 255))
        rgb_img.paste(img, mask=img.split()[3])
        ProcessedImages.append(rgb_img)
    
    temp_files = []
    for i, img in enumerate(ProcessedImages):
        temp_path = f"temp_{i}.jpg"
        img.save(temp_path, "JPEG")
        temp_files.append(temp_path)
    
    with open("out.pdf", "wb") as f:
        f.write(img2pdf.convert(temp_files))
    
    for temp_file in temp_files:
        os.remove(temp_file)

CreateTransparentPDF([GetImage() for _ in range(NUM_IMAGES)])
webbrowser.open_new_tab(f'file://{os.path.abspath("out.pdf")}')

True